# Introduction
Analysis of representation of languages in NYC

# Setup

## Dependencies

In [ ]:
%pip install pandas==2.3.1 pandasql==0.7.3

In [ ]:
import pandas as pd
import pandasql as ps

## Data Collection and Processing
See README for required data, and where it'll be expected

In [ ]:
lep = pd.read_csv(
    'data/lep.csv',
    names=['years', 'borough', 'districtCode', 'district', 'language', 'lepPopulation', 'lepPercent', 'cvalepPopulation', 'cvalepPercent'],
    header=0,
    quotechar='"',
    delimiter=','
)
lep.Name = "lep"
lep.describe(include='all')

In [ ]:
def clean_lang(lang_text):
    """TODO maybe better function name, this cleans up the text of a language name""" 
    return lang_text.split('(', 1)[0].strip().replace(' ', '-')

In [ ]:
# mapping "Borough Community District Code" to CDTA code - just eyeballing the names, the mapping is pretty obvious
def get_cdta_code(bcdc_code: str) -> str:
    borough_id, district_id = bcdc_code[0], bcdc_code[1:2+1]
    borough_code = {
        "1": "MN",
        "2": "BX",
        "3": "BK",
        "4": "QN",
        "5": "SI"
    }[borough_id]

    return f"{borough_code}{district_id}"

In [ ]:
lep['cdta'] = lep['districtCode'].astype('int').astype('str').apply(get_cdta_code)
lep.to_csv("data/lep_cdta.csv")
cdta = pd.read_csv("data/cdta.csv")
joined = pd.merge(cdta, lep, how='outer', left_on="CDTA2020", right_on="cdta")

In [ ]:
!mkdir -p data/languages

for lang in joined['language'].dropna().unique():
    title = clean_lang(lang)
    joined[joined['language'].str.casefold()==lang.lower()].to_csv(f'data/languages/{title}.csv')

# Analysis

In [ ]:
# Districts by Borough
# Note: districts mention multiple neighborhoods
districts_by_borough_df = ps.sqldf(f"""
    SELECT
        json_group_array(distinct json_object('district', district)) AS districts,
        borough
    FROM {lep.Name}
    GROUP BY borough
""")

import json
for _, districts, borough in districts_by_borough_df.to_records():
    print(
        f"{borough}: \n{'\n'.join("\t"+d['district'] for d in json.loads(districts))}"
    )

In [ ]:
language_counts_df = ps.sqldf(f"""
    SELECT language, SUM(cvalepPopulation) as total
    FROM {lep.Name}
    GROUP BY language
    ORDER BY total DESC
""")

In [ ]:
top_languages = language_counts_df['language'][:25].apply(clean_lang).to_list()
print(top_languages)

# QGIS Data Import Snippets
Run these in QGIS

In [ ]:
# copy files to windows desktop because QGIS lives in Windows on my machine
!cp -r data/languages /mnt/c/Users/Bieni/Desktop

In [ ]:
# NOTE: if running this code in the QGIS terminal, you'll have to copy some things like `top_languages`
import os
from qgis.core import (
    QgsVectorLayer,
    QgsProject
)

CWD = "C:/Users/Bieni/Desktop/languages"
os.chdir(CWD)

# NOTE: I DO NOT understand this naming structure, but it works! https://gis.stackexchange.com/a/147571/320861
for fn in os.listdir(CWD):
    lang_name = fn.replace('.csv', '')
    if lang_name not in top_languages: continue
    uri = f"file:///{CWD}/{fn}?type=csv&delimiter=,&useHeader=yes&wktField=the_geom&crs=epsg:4326"
    vlayer = QgsVectorLayer(uri, lang_name, "delimitedtext")
    QgsProject.instance().addMapLayer(vlayer)
    vlayer.setRenderer(
        QgsGraduatedSymbolRenderer.createRenderer(
            vlayer,
            "cvalepPopulation",
            5,
            QgsGraduatedSymbolRenderer.Mode.EqualInterval,
            QgsFillSymbol.createSimple({}),
            QgsGradientColorRamp(QColor.fromRgb(255, 255, 255), QColor.fromRgb(100, 255, 255))
        )
    )